In [10]:
import numpy as np
import pandas as pd
from IPython.display import clear_output
from matplotlib import pyplot as plt
import os
from scipy import sparse
import pickle

from art import config
from art.estimators.classification import BlackBoxClassifier
from art.attacks.evasion import CarliniL2Method, SaliencyMapMethod
from art.utils import to_categorical
from art.utils import load_dataset, get_file, compute_accuracy

In [2]:
os.chdir('/home/rcgonzal/DSC180Malware/m2v-adversarial-hindroid/')
!pwd

/home/rcgonzal/DSC180Malware/m2v-adversarial-hindroid


In [12]:
base_data = os.path.join('data/out/miniset-train')

In [3]:
def get_hindroid_features(app, base_data):
    if os.path.exists(app):
        apis = pd.read_csv(app, usecols=['api'], squeeze=True, dtype=str).unique()
    else:
        try:
            apis = pd.read_csv(os.path.join('data', 'out', 'all-apps', 'app-data', app+'.csv'), usecols=['api'], squeeze=True, dtype=str).unique()
        except FileNotFoundError as e:
            print("App not in app heap")
            raise e
    
    base_data_apis = pd.read_csv(os.path.join(base_data, 'api_map.csv'), index_col='api')
    
    A_row = pd.Series(np.zeros(base_data_apis.size), index=base_data_apis.index)
    A_row.loc[set(apis).intersection(base_data_apis.index)] = 1
    
    return A_row.values

predict(features):
    

In [7]:
x = get_hindroid_features('com.microsoft.todos', base_data='data/out/miniset-train')
x

array([1., 0., 0., ..., 0., 0., 0.])

In [9]:
x.to_sparse()

AttributeError: 'numpy.ndarray' object has no attribute 'to_sparse'

In [15]:
def ABA_predict(x):
    A = sparse.load_npz(os.path.join(base_data, 'hindroid', 'A_mat.npz'))
    B = sparse.load_npz(os.path.join(base_data, 'hindroid', 'B_mat.npz'))
    
    with open(os.path.join(base_data, 'hindroid', 'ABAT.mdl'), 'rb') as file:
        mdl = pickle.load(file)
    features = x * B * A.T
    return mdl.predict([features,])

ABA_predict(x)

array([0])

In [17]:
hindroid_blackbox = BlackBoxClassifier(ABA_predict, x.shape, 2, clip_values=(0, 1))

In [18]:
CarliniL2Method